#### > ### EEE4423: Deep Learning Lab

# Final Project: Long-tail Visual Recognition for Image Classification

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import os.path as osp
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import optuna
import joblib

from misc.project.utils import resnet18, IMBALANCECIFAR10, IMBALANCECIFAR100, compute_accuracy, IMBALANCECIFAR10_V, IMBALANCECIFAR100_V


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Find hyperparameter "Weight Decay"
#### (Simple example)
weight decay and accuracy are stored in a file 'WD'.

In [7]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]

BATCH_SIZE = 128

In [8]:
# num_cls = 10, 100 (CIFAR10, CIFAR100)
# val_num_each_cls = 1000, 100 (CIFAR10, CIFAR100)
# samples_of_each_cls = 5000, 500 (CIFAR10, CIFAR100)

def for_split_set(num_cls, val_num_each_cls, samples_of_each_cls):
    val_idx_list = []
    for i in range(num_cls):
        for j in range(val_num_each_cls):
            index = np.random.randint(i*0, (i+1)*samples_of_each_cls) # 0~4999, ..., 45000~49999(0~499, ..., 49500, 49999)의 숫자를 랜덤으로 하나 뽑는다.
            while index in val_idx_list: # index가 이미 리스트에 있다면 다시 뽑는다.
                index = np.random.randint(i*0, (i+1)*samples_of_each_cls)
            val_idx_list.append(index)

    train_idx = np.arange(50000)
    train_idx_list = list(np.delete(train_idx, val_idx_list))

    # Checking duplication
    for i in val_idx_list:
        if i in train_idx_list:
            print("중복이 있습니다!")
    
    return val_idx_list, train_idx_list 

In [9]:
def get_loaders():
    if DATASET == 'CIFAR10':
        val_idx, train_idx = for_split_set(10, 1000, 5000)
        train_dataset = IMBALANCECIFAR10_V(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, sub_idx=val_idx, train=True, download=True, transform=transform_train)
        val_dataset = IMBALANCECIFAR10_V(root='../dataset/project', imb_factor=1, train=True, download=True, sub_idx=train_idx, transform=transform_test)
    elif DATASET == 'CIFAR100':
        val_idx, train_idx = for_split_set(100, 100, 500)
        train_dataset = IMBALANCECIFAR100_V(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, sub_idx=val_idx, train=True, download=True, transform=transform_train)
        val_dataset = IMBALANCECIFAR100_V(root='../dataset/project', imb_factor=1, train=True, download=True, sub_idx=train_idx, transform=transform_test)

    cls_num_list = train_dataset.get_cls_num_list()
    print('cls num list(train_dataset):')
    print(cls_num_list)

    cls_num_list = val_dataset.get_cls_num_list()
    print('cls num list(val_dataset):')
    print(cls_num_list)
    num_classes = len(cls_num_list)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=4, drop_last=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=100, shuffle=False,
        num_workers=4, )

    return train_loader, val_loader, num_classes

In [10]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
        return pred

In [47]:
def train(EPOCHS, model, train_loader, criterion, optimizer, scheduler):

    for epoch in range(EPOCHS):
        loss_history = []
        model.train()
        for batch_index, data in enumerate(train_loader):
            image, target = data
            image, target = image.cuda(), target.cuda()

            pred = model(image)
            loss = criterion(pred, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_history.append(loss.item())

        topk_acc, head_acc, tail_acc = compute_accuracy(train_loader, model)
        loss_mean = np.mean(loss_history)
        scheduler.step()

        print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))

In [ ]:
def test(model, test_loader):
    model.eval()
    topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)
    print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

    accuracy = topk_acc[0]
    
    return accuracy

In [ ]:
LR = 0.1
MOMENTUM = 0.9
criterion = nn.CrossEntropyLoss()
torch.manual_seed(0)
train_loader, test_loader, num_classes = get_loaders()

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[4000, 2397, 1437, 861, 516, 309, 185, 111, 66, 40]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [ ]:
def train_model(trial):
    cfg = {
        'n_epoch': trial.suggest_categorical('n_epoch', [1,2,3]), # In the actual experiment, proceed to 90, 200
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
    }

    model = ResNet18(num_classes)
    model = model.cuda()

    optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=cfg['weight_decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

    train(cfg['n_epoch'], model, train_loader, criterion, optimizer, scheduler)
    test_accuracy = test(model, test_loader)
    
    return test_accuracy

In [ ]:
if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=3) # 실제 실험에서는 10 or 12로 진행
    joblib.dump(study, 'test_sampling.pkl')

[I 2022-06-18 12:27:17,959] A new study created in memory with name: no-name-d1495dd3-a601-4aaf-8d05-a36bd83cb572


Epoch: [001] 	 Loss 2.4846 	 Acc 37.19 	 AccHead 40.07 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5100 	 Acc 50.69 	 AccHead 54.62 	 AccTail 0.00


[I 2022-06-18 12:27:35,932] Trial 0 finished with value: 19.018281936645508 and parameters: {'n_epoch': 2, 'weight_decay': 0.003762899404711573}. Best is trial 0 with value: 19.018281936645508.


Acc 19.02 	 AccHead 37.66 	 AccTail 0.00
Epoch: [001] 	 Loss 2.7323 	 Acc 39.62 	 AccHead 42.68 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5564 	 Acc 54.60 	 AccHead 58.82 	 AccTail 0.00
Epoch: [003] 	 Loss 1.7739 	 Acc 56.29 	 AccHead 60.62 	 AccTail 0.00


[I 2022-06-18 12:28:01,464] Trial 1 finished with value: 22.593677520751953 and parameters: {'n_epoch': 3, 'weight_decay': 1.3695730368201831e-05}. Best is trial 1 with value: 22.593677520751953.


Acc 22.59 	 AccHead 44.74 	 AccTail 0.00
Epoch: [001] 	 Loss 2.4117 	 Acc 40.27 	 AccHead 43.38 	 AccTail 0.00
Epoch: [002] 	 Loss 1.3575 	 Acc 46.27 	 AccHead 49.83 	 AccTail 0.00
Epoch: [003] 	 Loss 1.3184 	 Acc 53.95 	 AccHead 58.11 	 AccTail 0.00


[I 2022-06-18 12:28:27,023] Trial 2 finished with value: 17.654783248901367 and parameters: {'n_epoch': 3, 'weight_decay': 0.030911012891264054}. Best is trial 1 with value: 22.593677520751953.


Acc 17.65 	 AccHead 34.96 	 AccTail 0.00


In [ ]:
study = joblib.load('test_sampling.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(3)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,19.018282,0 days 00:00:17.970055,2,0.003763
1,1,22.593678,0 days 00:00:25.529832,3,0.000014
2,2,17.654783,0 days 00:00:25.556325,3,0.030911


## Optimal Weight decay by sampling

#### Data representation
*  DATASET : CIFAR10 = 10, CIFAR100 = 10
*  IMB_TYPE : exp, step
*  IMB_FACTOR : 0.1, 0.01

(비슷한 수치()가 나왔을 때는 tail acc이 높은 것으로 선택)
#### Best Weight decay
* 10, exp, 0.1 = 0.000023 (90 epochs)
* 10, exp, 0.01 = 0.000042 (90 epochs)
* 10, step, 0.1 = 0.000052 (90 epochs)
* 10, step, 0.01 = 0.000288 (200 epochs)
* 100, exp, 0.1 = 0.001405 (200 epochs)
* 100, exp, 0.01 = 0.001043 (200 epochs)
* 100, step, 0.1 = 0.000479 (200 epochs)
* 100, step, 0.01 = 0.000670 (200 epochs)

##### If the integer units are the same, the higher tail accuracy is chosen.

# Find $τ$ value for $τ$-normalization   

In [11]:
# num_cls = 10, 100 (CIFAR10, CIFAR100)
# val_num_each_cls = 1000, 100 (CIFAR10, CIFAR100)
# samples_of_each_cls = 5000, 500 (CIFAR10, CIFAR100)

def for_split_set(num_cls, val_num_each_cls, samples_of_each_cls):
    val_idx_list = []
    for i in range(num_cls):
        for j in range(val_num_each_cls):
            index = np.random.randint(i*0, (i+1)*samples_of_each_cls) # 0~4999, ..., 45000~49999(0~499, ..., 49500, 49999)의 숫자를 랜덤으로 하나 뽑는다.
            while index in val_idx_list: # index가 이미 리스트에 있다면 다시 뽑는다.
                index = np.random.randint(i*0, (i+1)*samples_of_each_cls)
            val_idx_list.append(index)

    train_idx = np.arange(50000)
    train_idx_list = list(np.delete(train_idx, val_idx_list))

    # Checking duplication
    for i in val_idx_list:
        if i in train_idx_list:
            print("중복이 있습니다!")
    
    return val_idx_list, train_idx_list 

In [12]:
def get_loaders():
    if DATASET == 'CIFAR10':
        val_idx, train_idx = for_split_set(10, 1000, 5000)
        train_dataset = IMBALANCECIFAR10_V(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, sub_idx=val_idx, train=True, download=True, transform=transform_train)
        val_dataset = IMBALANCECIFAR10_V(root='../dataset/project', imb_factor=1, train=True, download=True, sub_idx=train_idx, transform=transform_test)
    elif DATASET == 'CIFAR100':
        val_idx, train_idx = for_split_set(100, 100, 500)
        train_dataset = IMBALANCECIFAR100_V(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, sub_idx=val_idx, train=True, download=True, transform=transform_train)
        val_dataset = IMBALANCECIFAR100_V(root='../dataset/project', imb_factor=1, train=True, download=True, sub_idx=train_idx, transform=transform_test)

    cls_num_list = train_dataset.get_cls_num_list()
    print('cls num list(train_dataset):')
    print(cls_num_list)

    cls_num_list = val_dataset.get_cls_num_list()
    print('cls num list(val_dataset):')
    print(cls_num_list)
    num_classes = len(cls_num_list)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=4, drop_last=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=100, shuffle=False,
        num_workers=4, )

    return train_dataset, train_loader, val_loader, num_classes

In [13]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
        return pred

In [14]:
def train(EPOCHS):
    for epoch in range(EPOCHS):
        loss_history = []
        model.train()
        for batch_index, data in enumerate(train_loader):
            image, target = data
            image, target = image.cuda(), target.cuda()

            pred = model(image)
            loss = criterion(pred, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_history.append(loss.item())

        topk_acc, head_acc, tail_acc = compute_accuracy(train_loader, model)
        loss_mean = np.mean(loss_history)
        scheduler.step()

        print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))
    
    print(epoch)
    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch},
        osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(EPOCHS))
    )

In [15]:
def test():
    model.eval()
    topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)
    # head : training sample개수가 많은 집함, tail : training samplg이 적은 아이들 -> head의 성능저하를 최소화하고 tail의 성능을 올리는 것이 목표
    print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

In [30]:
def testEeahClass():
    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            
            for i in range(len(labels)):  # batch size of test_loader
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    acc = []
    for i in range(num_classes):
        if class_total[i] == 0:
            acc.append(0)
            continue
        acc.append(100 * class_correct[i] / class_total[i])
    
    for i in range(num_classes):
        print('Accuracy of %5s : %2d %%' % (
            train_dataset.classes[i], acc[i]))
    
    
    return acc

In [17]:
tau_list = []

In [18]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/10-exp-0.1-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000023
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [20]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[4000, 3097, 2397, 1856, 1437, 1113, 861, 667, 516, 400]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 2.4303 	 Acc 42.27 	 AccHead 53.95 	 AccTail 0.25
Epoch: [002] 	 Loss 1.7202 	 Acc 44.50 	 AccHead 55.38 	 AccTail 5.32
Epoch: [003] 	 Loss 1.5470 	 Acc 49.56 	 AccHead 58.89 	 AccTail 16.00
Epoch: [004] 	 Loss 1.4517 	 Acc 50.27 	 AccHead 61.86 	 AccTail 8.64
Epoch: [005] 	 Loss 1.3919 	 Acc 51.77 	 AccHead 59.71 	 AccTail 23.24
Epoch: [006] 	 Loss 1.3240 	 Acc 55.20 	 AccHead 63.75 	 AccTail 24.45
Epoch: [007] 	 Loss 1.2898 	 Acc 56.48 	 AccHead 65.19 	 AccTail 25.18
Epoch: [008] 	 Loss 1.2461 	 Acc 57.41 	 AccHead 67.46 	 AccTail 21.26
Epoch: [009] 	 Loss 1.2150 	 Acc 58.64 	 AccHead 65.31 	 AccTail 34.61
Epoch: [010] 	 Loss 1.1734 	 Acc 60.57 	 AccHead 67.40 	 AccTail 36.02
Epoch: [011] 	 Loss 1.1361 	 Acc 61.42 	 Ac

In [21]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/10-exp-0.01-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000042
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [22]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[4000, 2397, 1437, 861, 516, 309, 185, 111, 66, 40]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 2.9861 	 Acc 44.00 	 AccHead 47.40 	 AccTail 0.00
Epoch: [002] 	 Loss 1.8592 	 Acc 47.37 	 AccHead 51.03 	 AccTail 0.00
Epoch: [003] 	 Loss 1.4960 	 Acc 43.14 	 AccHead 46.47 	 AccTail 0.00
Epoch: [004] 	 Loss 1.3264 	 Acc 57.01 	 AccHead 61.41 	 AccTail 0.00
Epoch: [005] 	 Loss 1.2236 	 Acc 58.21 	 AccHead 62.72 	 AccTail 0.00
Epoch: [006] 	 Loss 1.1949 	 Acc 60.91 	 AccHead 65.61 	 AccTail 0.00
Epoch: [007] 	 Loss 1.1515 	 Acc 61.92 	 AccHead 66.69 	 AccTail 0.00
Epoch: [008] 	 Loss 1.1090 	 Acc 63.38 	 AccHead 68.27 	 AccTail 0.00
Epoch: [009] 	 Loss 1.0910 	 Acc 62.07 	 AccHead 66.87 	 AccTail 0.00
Epoch: [010] 	 Loss 1.0654 	 Acc 65.97 	 AccHead 71.02 	 AccTail 0.71
Epoch: [011] 	 Loss 1.0377 	 Acc 66.21 	 AccHead 71.34 

In [23]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/10-step-0.1-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000052
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [24]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[4000, 4000, 4000, 4000, 4000, 400, 400, 400, 400, 400]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 2.3790 	 Acc 32.01 	 AccHead 35.05 	 AccTail 0.42
Epoch: [002] 	 Loss 1.6655 	 Acc 46.33 	 AccHead 50.79 	 AccTail 0.00
Epoch: [003] 	 Loss 1.4529 	 Acc 51.42 	 AccHead 56.37 	 AccTail 0.00
Epoch: [004] 	 Loss 1.3541 	 Acc 52.02 	 AccHead 57.03 	 AccTail 0.00
Epoch: [005] 	 Loss 1.3000 	 Acc 55.58 	 AccHead 60.94 	 AccTail 0.00
Epoch: [006] 	 Loss 1.2533 	 Acc 57.78 	 AccHead 63.30 	 AccTail 0.42
Epoch: [007] 	 Loss 1.1986 	 Acc 58.65 	 AccHead 64.26 	 AccTail 0.47
Epoch: [008] 	 Loss 1.1696 	 Acc 61.08 	 AccHead 66.90 	 AccTail 0.62
Epoch: [009] 	 Loss 1.1185 	 Acc 62.07 	 AccHead 67.92 	 AccTail 1.30
Epoch: [010] 	 Loss 1.0778 	 Acc 65.24 	 AccHead 71.43 	 AccTail 0.78
Epoch: [011] 	 Loss 1.0408 	 Acc 63.99 	 AccHead 70

In [25]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/10-step-0.01-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000288
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [26]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[4000, 4000, 4000, 4000, 4000, 40, 40, 40, 40, 40]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 2.1632 	 Acc 37.66 	 AccHead 37.96 	 AccTail 0.00
Epoch: [002] 	 Loss 1.3774 	 Acc 52.71 	 AccHead 53.14 	 AccTail 0.00
Epoch: [003] 	 Loss 1.1608 	 Acc 57.45 	 AccHead 57.92 	 AccTail 0.00
Epoch: [004] 	 Loss 1.0665 	 Acc 56.93 	 AccHead 57.38 	 AccTail 0.00
Epoch: [005] 	 Loss 1.0026 	 Acc 61.13 	 AccHead 61.62 	 AccTail 0.00
Epoch: [006] 	 Loss 0.9510 	 Acc 60.30 	 AccHead 60.79 	 AccTail 0.00
Epoch: [007] 	 Loss 0.9024 	 Acc 68.34 	 AccHead 68.89 	 AccTail 0.00
Epoch: [008] 	 Loss 0.8420 	 Acc 69.78 	 AccHead 70.35 	 AccTail 0.00
Epoch: [009] 	 Loss 0.8027 	 Acc 72.70 	 AccHead 73.29 	 AccTail 0.00
Epoch: [010] 	 Loss 0.7523 	 Acc 73.50 	 AccHead 74.09 	 AccTail 0.00
Epoch: [011] 	 Loss 0.7227 	 Acc 75.05 	 AccHead 75.66 	

In [27]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/100-exp-0.1-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.001405
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [28]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[400, 390, 381, 373, 364, 356, 347, 339, 332, 324, 316, 309, 302, 295, 288, 282, 275, 269, 263, 257, 251, 245, 239, 234, 228, 223, 218, 213, 208, 203, 199, 194, 190, 185, 181, 177, 173, 169, 165, 161, 157, 154, 150, 147, 143, 140, 137, 134, 130, 127, 125, 122, 119, 116, 113, 111, 108, 106, 103, 101, 99, 96, 94, 92, 90, 88, 86, 84, 82, 80, 78, 76, 74, 73, 71, 69, 68, 66, 65, 63, 62, 60, 59, 58, 56, 55, 54, 52, 51, 50, 49, 48, 47, 45, 44, 43, 42, 41, 40, 40]
cls num list(val_dataset):
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1

ZeroDivisionError: float division by zero

In [38]:
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Accuracy of apple :  1 %
Accuracy of aquarium_fish :  4 %
Accuracy of  baby :  5 %
Accuracy of  bear :  7 %
Accuracy of beaver :  5 %
Accuracy of   bed :  3 %
Accuracy of   bee :  4 %
Accuracy of beetle :  5 %
Accuracy of bicycle :  9 %
Accuracy of bottle :  9 %
Accuracy of  bowl :  4 %
Accuracy of   boy :  5 %
Accuracy of bridge :  6 %
Accuracy of   bus :  6 %
Accuracy of butterfly :  5 %
Accuracy of camel :  4 %
Accuracy of   can :  3 %
Accuracy of castle :  4 %
Accuracy of caterpillar :  3 %
Accuracy of cattle :  4 %
Accuracy of chair :  5 %
Accuracy of chimpanzee :  2 %
Accuracy of clock :  3 %
Accuracy of cloud :  1 %
Accuracy of cockroach :  7 %
Accuracy of couch :  6 %
Accuracy of  crab :  4 %
Accuracy of crocodile :  1 %
Accuracy of   cup :  4 %
Accuracy of dinosaur :  3 %
Accuracy of dolphin :  1 %
Accuracy of elephant :  0 %
Accuracy of flatfish :  1 %
Accuracy of forest :  0 %
Accuracy of   fox :  1 %
Accuracy of  girl :  2 %
Accuracy of hamster :  1 %
Accuracy of house :  1

In [40]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/100-exp-0.01-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.001043
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [41]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[400, 381, 364, 347, 332, 316, 302, 288, 275, 263, 251, 239, 228, 218, 208, 199, 190, 181, 173, 165, 157, 150, 143, 137, 130, 125, 119, 113, 108, 103, 99, 94, 90, 86, 82, 78, 74, 71, 68, 65, 62, 59, 56, 54, 51, 49, 47, 44, 42, 40, 39, 37, 35, 33, 32, 30, 29, 28, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 16, 15, 14, 14, 13, 12, 12, 11, 11, 10, 10, 9, 9, 8, 8, 8, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 4, 4, 4, 4, 4]
cls num list(val_dataset):
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1

In [42]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/100-step-0.1-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000479
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [43]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40]
cls num list(val_dataset):
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 1

In [44]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/100-step-0.01-find_tau' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000670
EPOCHS = 80

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [45]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[4000, 4000, 4000, 4000, 4000, 40, 40, 40, 40, 40]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 2.2103 	 Acc 34.07 	 AccHead 28.44 	 AccTail 55.90
Epoch: [002] 	 Loss 1.3190 	 Acc 50.48 	 AccHead 47.97 	 AccTail 60.20
Epoch: [003] 	 Loss 1.1541 	 Acc 57.66 	 AccHead 56.60 	 AccTail 61.79
Epoch: [004] 	 Loss 1.0652 	 Acc 58.89 	 AccHead 55.73 	 AccTail 71.17
Epoch: [005] 	 Loss 1.0092 	 Acc 64.00 	 AccHead 67.05 	 AccTail 52.17
Epoch: [006] 	 Loss 0.9409 	 Acc 62.83 	 AccHead 61.27 	 AccTail 68.88
Epoch: [007] 	 Loss 0.8846 	 Acc 68.49 	 AccHead 71.82 	 AccTail 55.56
Epoch: [008] 	 Loss 0.8329 	 Acc 68.68 	 AccHead 72.01 	 AccTail 55.76
Epoch: [009] 	 Loss 0.7897 	 Acc 72.26 	 AccHead 73.49 	 AccTail 67.49
Epoch: [010] 	 Loss 0.7600 	 Acc 71.80 	 AccHead 76.95 	 AccTail 51.82
Epoch: [011] 	 Loss 0.7318 	 Acc 72.30 	 AccHe

KeyboardInterrupt: 

In [46]:
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()
tau = np.mean(np.array(avg_accu)) / 100
print(tau)
tau_list.append(tau)

Acc 41.38 	 AccHead 81.92 	 AccTail 0.04
----------------------------
get the top-1 accuracy for each of the classes.
Accuracy of airplane : 89 %
Accuracy of automobile : 91 %
Accuracy of  bird : 81 %
Accuracy of   cat : 70 %
Accuracy of  deer : 75 %
Accuracy of   dog :  0 %
Accuracy of  frog :  0 %
Accuracy of horse :  0 %
Accuracy of  ship :  0 %
Accuracy of truck :  0 %
0.40980000000000005


In [47]:
tau_list

[0.67859012345679,
 0.49658367346938775,
 0.6087934655775963,
 0.44349999999999995,
 0.0175,
 0.07400000000000001,
 0.1106,
 0.40980000000000005]

In [98]:
tau_dict = {'10_exp_1': 0.67859012345679, '10_exp_01':0.49658367346938775, '10_step_1': 0.6087934655775963, '10_step_01': 0.44349999999999995, '100_exp_1': 0.0175, '100_exp_01': 0.07400000000000001, '100_step_1': 0.1106, '100_step_01': 0.40980000000000005}

In [99]:
file1 = open("tau.txt", "w") 
str1 = repr(tau_dict)
file1.write("dict1 = " + str1 + "\n")
file1.close()
 
f = open('tau.txt', 'r')
if f.mode=='r':
    contents= f.read()

# Train

## Base model (no weight decay) and Base model + $\tau$-normalization

In [48]:
def get_loaders():
    if DATASET == 'CIFAR10':
        train_dataset = IMBALANCECIFAR10(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
        test_dataset = torchvision.datasets.CIFAR10(root='../dataset/project', train=False, download=True, transform=transform_test)
    elif DATASET == 'CIFAR100':
        train_dataset = IMBALANCECIFAR100(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
        test_dataset = torchvision.datasets.CIFAR100(root='../dataset/project', train=False, download=True, transform=transform_test)

    cls_num_list = train_dataset.get_cls_num_list()
    print('cls num list:')
    print(cls_num_list)
    num_classes = len(cls_num_list)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=4, drop_last=True)

    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=100, shuffle=False,
        num_workers=4, )

    return train_dataset, train_loader, test_loader, num_classes

In [49]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)


    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
        return pred

In [50]:
def train(EPOCHS):
    for epoch in range(EPOCHS):
        loss_history = []
        model.train()
        for batch_index, data in enumerate(train_loader):
            image, target = data
            image, target = image.cuda(), target.cuda()

            pred = model(image)
            loss = criterion(pred, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_history.append(loss.item())

        topk_acc, head_acc, tail_acc = compute_accuracy(train_loader, model)
        loss_mean = np.mean(loss_history)
        scheduler.step()

        print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))
    
    print(epoch)
    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch},
        osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(EPOCHS))
    )

In [51]:
def test():
    model.eval()
    topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)
    # head : training sample개수가 많은 집함, tail : training samplg이 적은 아이들 -> head의 성능저하를 최소화하고 tail의 성능을 올리는 것이 목표
    print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

In [52]:
def testEeahClass():
    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):  # batch size of test_loader
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    acc = []
    for i in range(num_classes):
        if class_total[i] == 0:
            acc.append(0)
            continue
        acc.append(100 * class_correct[i] / class_total[i])
    
    for i in range(num_classes):
        print('Accuracy of %5s : %2d %%' % (
            train_dataset.classes[i], acc[i]))
    
    
    return acc

In [55]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/10-exp-0.1-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [ ]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

In [57]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[0] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 3871, 2997, 2320, 1796, 1391, 1077, 834, 645, 500]
Acc 71.72 	 AccHead 76.40 	 AccTail 67.04
----------------------------
get the top-1 accuracy for each of the classes.
Acc 73.47 	 AccHead 74.76 	 AccTail 72.18
Accuracy of airplane : 85 %
Accuracy of automobile : 93 %
Accuracy of  bird : 73 %
Accuracy of   cat : 54 %
Accuracy of  deer : 67 %
Accuracy of   dog : 64 %
Accuracy of  frog : 76 %
Accuracy of horse : 74 %
Accuracy of  ship : 76 %
Accuracy of truck : 68 %


In [70]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/10-exp-0.01-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [71]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[1] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 2997, 1796, 1077, 645, 387, 232, 139, 83, 50]
Acc 50.84 	 AccHead 72.88 	 AccTail 28.80
----------------------------
get the top-1 accuracy for each of the classes.
Acc 55.03 	 AccHead 71.54 	 AccTail 38.52
Accuracy of airplane : 89 %
Accuracy of automobile : 92 %
Accuracy of  bird : 62 %
Accuracy of   cat : 55 %
Accuracy of  deer : 57 %
Accuracy of   dog : 49 %
Accuracy of  frog : 51 %
Accuracy of horse : 33 %
Accuracy of  ship : 43 %
Accuracy of truck : 15 %


In [72]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/10-step-0.1-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [73]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[2] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 5000, 5000, 5000, 5000, 500, 500, 500, 500, 500]
Acc 68.02 	 AccHead 85.38 	 AccTail 50.66
----------------------------
get the top-1 accuracy for each of the classes.
Acc 72.51 	 AccHead 82.66 	 AccTail 62.36
Accuracy of airplane : 83 %
Accuracy of automobile : 94 %
Accuracy of  bird : 75 %
Accuracy of   cat : 76 %
Accuracy of  deer : 84 %
Accuracy of   dog : 41 %
Accuracy of  frog : 61 %
Accuracy of horse : 64 %
Accuracy of  ship : 75 %
Accuracy of truck : 68 %


In [74]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/10-step-0.01-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [75]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[3] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 5000, 5000, 5000, 5000, 50, 50, 50, 50, 50]
Acc 49.11 	 AccHead 87.66 	 AccTail 10.56
----------------------------
get the top-1 accuracy for each of the classes.
Acc 54.42 	 AccHead 87.02 	 AccTail 21.82
Accuracy of airplane : 87 %
Accuracy of automobile : 96 %
Accuracy of  bird : 82 %
Accuracy of   cat : 82 %
Accuracy of  deer : 85 %
Accuracy of   dog :  4 %
Accuracy of  frog : 22 %
Accuracy of horse : 29 %
Accuracy of  ship : 37 %
Accuracy of truck : 15 %


In [76]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/100-exp-0.1-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [77]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[4] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 488, 477, 466, 455, 445, 434, 424, 415, 405, 396, 387, 378, 369, 361, 352, 344, 336, 328, 321, 314, 306, 299, 292, 286, 279, 273, 266, 260, 254, 248, 243, 237, 232, 226, 221, 216, 211, 206, 201, 197, 192, 188, 183, 179, 175, 171, 167, 163, 159, 156, 152, 149, 145, 142, 139, 135, 132, 129, 126, 123, 121, 118, 115, 112, 110, 107, 105, 102, 100, 98, 95, 93, 91, 89, 87, 85, 83, 81, 79, 77, 75, 74, 72, 70, 69, 67, 66, 64, 63, 61, 60, 58, 57, 56, 54, 53, 52, 51, 50]
Acc 38.57 	 AccHead 48.10 	 AccTail 29.04
----------------------------
get the top-1 accuracy for each of the classes.
Acc 40.04 	 AccHead 43.90 	 AccTail 36.18
Accuracy of apple : 81 %
Accuracy of aquarium_fish : 68 %
Accuracy of  baby : 48 %
Accuracy of  bear : 27 %
Accuracy of beaver : 33 %
Accuracy of   bed : 51 %
Accuracy of   bee : 50 %
Accuracy of beetle : 56 %
Accuracy of bicycle : 66 %
Accuracy of bottle : 59 %
Accuracy of  bo

In [78]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/100-exp-0.01-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [79]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[5] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 477, 455, 434, 415, 396, 378, 361, 344, 328, 314, 299, 286, 273, 260, 248, 237, 226, 216, 206, 197, 188, 179, 171, 163, 156, 149, 142, 135, 129, 123, 118, 112, 107, 102, 98, 93, 89, 85, 81, 77, 74, 70, 67, 64, 61, 58, 56, 53, 51, 48, 46, 44, 42, 40, 38, 36, 35, 33, 32, 30, 29, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 15, 14, 13, 13, 12, 12, 11, 11, 10, 10, 9, 9, 8, 8, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5]
Acc 26.90 	 AccHead 41.80 	 AccTail 12.00
----------------------------
get the top-1 accuracy for each of the classes.
Acc 27.34 	 AccHead 31.62 	 AccTail 23.06
Accuracy of apple : 70 %
Accuracy of aquarium_fish : 55 %
Accuracy of  baby : 28 %
Accuracy of  bear : 20 %
Accuracy of beaver : 28 %
Accuracy of   bed : 33 %
Accuracy of   bee : 38 %
Accuracy of beetle : 34 %
Accuracy of bicycle : 40 %
Accuracy of bottle : 52 %
Accuracy of  bowl : 11 %
Accuracy of   boy : 21 %
Accuracy of bri

In [80]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/100-step-0.1-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [81]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[6] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
Acc 37.98 	 AccHead 56.44 	 AccTail 19.52
----------------------------
get the top-1 accuracy for each of the classes.
Acc 40.92 	 AccHead 48.54 	 AccTail 33.30
Accuracy of apple : 77 %
Accuracy of aquarium_fish : 67 %
Accuracy of  baby : 38 %
Accuracy of  bear : 21 %
Accuracy of beaver : 28 %
Accuracy of   bed : 49 %
Accuracy of   bee : 43 %
Accuracy of beetle : 50 %
Accuracy of bicycle : 65 %
Accuracy of bottle : 57 %
Accuracy of  bowl : 18 %
Accuracy o

In [82]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/100-step-0.01-base' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [85]:
#여ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ기ㅣㅣㅣㅣㅣㅣㅣㅣ##########
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()

PATH = f'{SAVE_DIR}/ep{EPOCHS:03d}.pth'
model.load_state_dict(torch.load(PATH)["model"])

test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[7] 

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Acc 29.49 	 AccHead 57.10 	 AccTail 1.88
----------------------------
get the top-1 accuracy for each of the classes.
Acc 22.18 	 AccHead 19.58 	 AccTail 24.78
Accuracy of apple : 26 %
Accuracy of aquarium_fish : 19 %
Accuracy of  baby : 12 %
Accuracy of  bear :  2 %
Accuracy of beaver :  4 %
Accuracy of   bed : 25 %
Accuracy of   bee : 10 %
Accuracy of beetle :  4 %
Accuracy of bicycle : 38 %
Accuracy of bottle : 26 %
Accuracy of  bowl :  5 %
Accuracy of   boy :  6 %
Accuracy of bridge : 12 %
Accuracy o

## Weight decay + tau-norm

In [53]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/10-exp-0.1' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000023
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [54]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')

print(tau_list[0])
tau_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
tau_norm *= tau_norm**tau_list[0]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(tau_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf")) # set 0

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 3871, 2997, 2320, 1796, 1391, 1077, 834, 645, 500]
Epoch: [001] 	 Loss 2.2998 	 Acc 42.92 	 AccHead 54.80 	 AccTail 0.20
Epoch: [002] 	 Loss 1.5921 	 Acc 47.68 	 AccHead 57.71 	 AccTail 11.64
Epoch: [003] 	 Loss 1.4587 	 Acc 51.78 	 AccHead 61.78 	 AccTail 15.81
Epoch: [004] 	 Loss 1.3611 	 Acc 53.41 	 AccHead 61.25 	 AccTail 25.27
Epoch: [005] 	 Loss 1.2992 	 Acc 55.78 	 AccHead 66.57 	 AccTail 16.98
Epoch: [006] 	 Loss 1.2349 	 Acc 56.99 	 AccHead 66.90 	 AccTail 21.26
Epoch: [007] 	 Loss 1.1832 	 Acc 60.87 	 AccHead 70.21 	 AccTail 27.25
Epoch: [008] 	 Loss 1.1181 	 Acc 60.20 	 AccHead 66.15 	 AccTail 38.77
Epoch: [009] 	 Loss 1.0701 	 Acc 63.95 	 AccHead 70.83 	 AccTail 39.18
Epoch: [010] 	 Loss 1.0169 	 Acc 66.34 	 AccHead 72.54 	 AccTail 44.09
Epoch: [011] 	 Loss 0.9777 	 Acc 66.64 	 AccHead 73.39 	 AccTail 42.38
Epoch: [012] 	 Loss 0.9439 	 Acc 69.30 	 AccHead 75.77 	 AccTail 46.05
E

In [60]:
 

DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/10-exp-0.01' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000042
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [61]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

print(tau_list[1])
n2_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
n2_norm *= n2_norm**tau_list[1]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(n2_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf"))

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 2997, 1796, 1077, 645, 387, 232, 139, 83, 50]
Epoch: [001] 	 Loss 2.7194 	 Acc 50.17 	 AccHead 54.06 	 AccTail 0.00
Epoch: [002] 	 Loss 1.4019 	 Acc 57.68 	 AccHead 62.15 	 AccTail 0.00
Epoch: [003] 	 Loss 1.2421 	 Acc 62.51 	 AccHead 67.35 	 AccTail 0.00
Epoch: [004] 	 Loss 1.1675 	 Acc 63.19 	 AccHead 68.08 	 AccTail 0.00
Epoch: [005] 	 Loss 1.1463 	 Acc 64.34 	 AccHead 69.36 	 AccTail 0.00
Epoch: [006] 	 Loss 1.0543 	 Acc 64.63 	 AccHead 69.65 	 AccTail 0.00
Epoch: [007] 	 Loss 1.0136 	 Acc 67.84 	 AccHead 73.08 	 AccTail 0.00
Epoch: [008] 	 Loss 0.9770 	 Acc 67.51 	 AccHead 72.69 	 AccTail 0.79
Epoch: [009] 	 Loss 0.9483 	 Acc 68.57 	 AccHead 73.75 	 AccTail 1.81
Epoch: [010] 	 Loss 0.9176 	 Acc 70.26 	 AccHead 75.66 	 AccTail 0.00
Epoch: [011] 	 Loss 0.9112 	 Acc 70.47 	 AccHead 75.89 	 AccTail 0.68
Epoch: [012] 	 Loss 0.8984 	 Acc 71.00 	 AccHead 75.39 	 AccTail 14.27
Epoch: [013] 	 L

In [86]:
 

DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/10-step-0.1' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000052
EPOCHS = 90

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [87]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

print(tau_list[2])
n2_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
n2_norm *= n2_norm**tau_list[2]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(n2_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf"))

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 5000, 5000, 5000, 5000, 500, 500, 500, 500, 500]
Epoch: [001] 	 Loss 2.2539 	 Acc 44.46 	 AccHead 48.90 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5034 	 Acc 48.45 	 AccHead 53.30 	 AccTail 0.00
Epoch: [003] 	 Loss 1.3633 	 Acc 54.46 	 AccHead 59.88 	 AccTail 0.28
Epoch: [004] 	 Loss 1.2649 	 Acc 57.37 	 AccHead 62.97 	 AccTail 1.41
Epoch: [005] 	 Loss 1.1891 	 Acc 59.11 	 AccHead 64.47 	 AccTail 5.54
Epoch: [006] 	 Loss 1.1201 	 Acc 62.12 	 AccHead 67.62 	 AccTail 7.14
Epoch: [007] 	 Loss 1.0637 	 Acc 64.75 	 AccHead 70.71 	 AccTail 5.14
Epoch: [008] 	 Loss 1.0029 	 Acc 65.17 	 AccHead 70.64 	 AccTail 10.44
Epoch: [009] 	 Loss 0.9593 	 Acc 67.47 	 AccHead 72.81 	 AccTail 14.00
Epoch: [010] 	 Loss 0.9025 	 Acc 70.52 	 AccHead 75.77 	 AccTail 17.99
Epoch: [011] 	 Loss 0.8661 	 Acc 71.49 	 AccHead 77.28 	 AccTail 13.63
Epoch: [012] 	 Loss 0.8325 	 Acc 73.70 	 AccHead 78.44 	 AccTail 26.38
Epoch: [0

In [88]:
 

DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/10-step-0.01' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000288
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [89]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

print(tau_list[3])
n2_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
n2_norm *= n2_norm**tau_list[3]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(n2_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf"))

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 5000, 5000, 5000, 5000, 50, 50, 50, 50, 50]
Epoch: [001] 	 Loss 2.1273 	 Acc 45.65 	 AccHead 46.11 	 AccTail 0.00
Epoch: [002] 	 Loss 1.2457 	 Acc 53.43 	 AccHead 53.96 	 AccTail 0.00
Epoch: [003] 	 Loss 1.0876 	 Acc 56.98 	 AccHead 57.55 	 AccTail 0.00
Epoch: [004] 	 Loss 1.0122 	 Acc 63.42 	 AccHead 64.05 	 AccTail 0.00
Epoch: [005] 	 Loss 0.9377 	 Acc 61.25 	 AccHead 61.86 	 AccTail 0.00
Epoch: [006] 	 Loss 0.8685 	 Acc 67.14 	 AccHead 67.81 	 AccTail 0.00
Epoch: [007] 	 Loss 0.8109 	 Acc 73.04 	 AccHead 73.77 	 AccTail 0.00
Epoch: [008] 	 Loss 0.7571 	 Acc 73.84 	 AccHead 74.58 	 AccTail 0.00
Epoch: [009] 	 Loss 0.7127 	 Acc 75.98 	 AccHead 76.74 	 AccTail 0.00
Epoch: [010] 	 Loss 0.6739 	 Acc 76.38 	 AccHead 77.14 	 AccTail 0.00
Epoch: [011] 	 Loss 0.6617 	 Acc 77.66 	 AccHead 78.44 	 AccTail 0.00
Epoch: [012] 	 Loss 0.6432 	 Acc 78.60 	 AccHead 79.38 	 AccTail 0.00
Epoch: [013] 	 Loss

In [90]:
 

DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/100-exp-0.1' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.001405
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [91]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

print(tau_list[4])
n2_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
n2_norm *= n2_norm**tau_list[4]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(n2_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf"))

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 488, 477, 466, 455, 445, 434, 424, 415, 405, 396, 387, 378, 369, 361, 352, 344, 336, 328, 321, 314, 306, 299, 292, 286, 279, 273, 266, 260, 254, 248, 243, 237, 232, 226, 221, 216, 211, 206, 201, 197, 192, 188, 183, 179, 175, 171, 167, 163, 159, 156, 152, 149, 145, 142, 139, 135, 132, 129, 126, 123, 121, 118, 115, 112, 110, 107, 105, 102, 100, 98, 95, 93, 91, 89, 87, 85, 83, 81, 79, 77, 75, 74, 72, 70, 69, 67, 66, 64, 63, 61, 60, 58, 57, 56, 54, 53, 52, 51, 50]
Epoch: [001] 	 Loss 4.4929 	 Acc 10.41 	 AccHead 13.57 	 AccTail 0.26
Epoch: [002] 	 Loss 3.7510 	 Acc 14.08 	 AccHead 17.14 	 AccTail 4.25
Epoch: [003] 	 Loss 3.5156 	 Acc 17.39 	 AccHead 20.96 	 AccTail 5.96
Epoch: [004] 	 Loss 3.3513 	 Acc 18.56 	 AccHead 22.98 	 AccTail 4.37
Epoch: [005] 	 Loss 3.1995 	 Acc 22.78 	 AccHead 27.64 	 AccTail 7.19
Epoch: [006] 	 Loss 3.0941 	 Acc 25.88 	 AccHead 31.20 	 AccTail 8.75
Epoch: [007] 	 Loss

In [92]:
 

DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/100-exp-0.01' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.001043
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [93]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

print(tau_list[5])
n2_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
n2_norm *= n2_norm**tau_list[5]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(n2_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf"))

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 477, 455, 434, 415, 396, 378, 361, 344, 328, 314, 299, 286, 273, 260, 248, 237, 226, 216, 206, 197, 188, 179, 171, 163, 156, 149, 142, 135, 129, 123, 118, 112, 107, 102, 98, 93, 89, 85, 81, 77, 74, 70, 67, 64, 61, 58, 56, 53, 51, 48, 46, 44, 42, 40, 38, 36, 35, 33, 32, 30, 29, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 15, 14, 13, 13, 12, 12, 11, 11, 10, 10, 9, 9, 8, 8, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5]
Epoch: [001] 	 Loss 4.2559 	 Acc 13.72 	 AccHead 15.03 	 AccTail 0.00
Epoch: [002] 	 Loss 3.5742 	 Acc 17.48 	 AccHead 19.13 	 AccTail 0.11
Epoch: [003] 	 Loss 3.3653 	 Acc 22.06 	 AccHead 24.03 	 AccTail 1.49
Epoch: [004] 	 Loss 3.1752 	 Acc 22.94 	 AccHead 25.07 	 AccTail 0.64
Epoch: [005] 	 Loss 3.0355 	 Acc 26.57 	 AccHead 28.94 	 AccTail 1.81
Epoch: [006] 	 Loss 2.9157 	 Acc 27.19 	 AccHead 29.49 	 AccTail 3.18
Epoch: [007] 	 Loss 2.8211 	 Acc 30.06 	 AccHead 32.50 	 AccTail 4.48

In [94]:
 

DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/100-step-0.1' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000479
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [95]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

print(tau_list[6])
n2_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
n2_norm *= n2_norm**tau_list[6]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(n2_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf"))

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
Epoch: [001] 	 Loss 3.9900 	 Acc 15.70 	 AccHead 17.27 	 AccTail 0.00
Epoch: [002] 	 Loss 3.3789 	 Acc 22.13 	 AccHead 24.31 	 AccTail 0.44
Epoch: [003] 	 Loss 3.1052 	 Acc 24.58 	 AccHead 27.01 	 AccTail 0.24
Epoch: [004] 	 Loss 2.9250 	 Acc 26.19 	 AccHead 28.76 	 AccTail 0.52
Epoch: [005] 	 Loss 2.7731 	 Acc 32.07 	 AccHead 35.11 	 AccTail 1.65
Epoch: [006] 	 Loss 2.6449 	 Acc 31.82 	 AccHead 34.75 	 AccTail 2.57
Epoch: [007] 	 Loss 2.5182 	 Acc 32.07 

In [96]:
 

DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/100-step-0.01' 

LR = 0.1
BATCH_SIZE = 128
MOMENTUM = 0.9
WEIGHT_DECAY = 0.000670
EPOCHS = 200

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [97]:
train_dataset, train_loader, test_loader, num_classes = get_loaders()
model = ResNet18(num_classes)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

# Let's build our model
train(EPOCHS)
print('Finished Training')
test() 
print('----------------------------')
print('get the top-1 accuracy for each of the classes.')
avg_accu = testEeahClass()

print(tau_list[7])
n2_norm = torch.norm(model.classifier.weight, p=2.0, dim=1)
n2_norm *= n2_norm**tau_list[7]

with torch.no_grad():
    for i in range(num_classes):
        model.classifier.weight[i] = model.classifier.weight[i].div_(n2_norm[i])
        model.classifier.bias[i] = model.classifier.bias[i].div_(float("inf"))

test()
avg_accu = testEeahClass()


torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': EPOCHS},
    osp.join(SAVE_DIR, 'tau-ep{:03d}.pth'.format(EPOCHS))
)

Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Epoch: [001] 	 Loss 3.7605 	 Acc 14.38 	 AccHead 14.53 	 AccTail 0.00
Epoch: [002] 	 Loss 3.1841 	 Acc 21.39 	 AccHead 21.61 	 AccTail 0.00
Epoch: [003] 	 Loss 2.9182 	 Acc 24.95 	 AccHead 25.20 	 AccTail 0.00
Epoch: [004] 	 Loss 2.7444 	 Acc 27.00 	 AccHead 27.27 	 AccTail 0.00
Epoch: [005] 	 Loss 2.5812 	 Acc 32.54 	 AccHead 32.86 	 AccTail 0.00
Epoch: [006] 	 Loss 2.4432 	 Acc 33.46 	 AccHead 33.79 	 AccTail 0.00
Epoch: [007] 	 Loss 2.3258 	 Acc 36.15 	 AccHead 36.51 	 AccTail 0.00
Epoch: [008] 	 Loss